In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import string
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import cv2
import multiprocessing as mp
import joblib

train_on_gpu = torch.cuda.is_available()
if torch.cuda.device_count() > 1:
    multiple_gpus = True
    
print(f"Using {torch.cuda.device_count()} GPUs")

torch.manual_seed(0)

Using 4 GPUs


In [2]:
le = joblib.load('../models/genre_encoder.sklearn')
df = pd.read_csv('../data/clean_book_data.csv')
df = df[['index', 'genre']]
classes = df.genre.unique()

In [3]:
train_transforms = transforms.Compose([transforms.ToPILImage(),
                                       transforms.Resize((224, 224)),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.ToPILImage(),
                                     transforms.Resize((224, 224)),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])



In [4]:
train_df, test_df = train_test_split(df, stratify = df.genre, test_size=.4, random_state=0)
val_df, test_df = train_test_split(test_df, stratify = test_df.genre, test_size=.5, random_state=0)

In [5]:
class DataSet(torch.utils.data.Dataset):
    def __init__(self, labels, data_directory, transform=None):
        super().__init__()
        self.labels = labels.values
        self.data_dir = data_directory
        self.transform=transform
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, index):
        name, label = self.labels[index]
        img_path = os.path.join(self.data_dir, name)
        img = cv2.imread(img_path)
        
        if self.transform is not None:
            img = self.transform(img)
        return img, label

In [6]:
batch_size = 196

train_data = DataSet(train_df,'../data/goodreads-best-books/images/images/', transform = train_transforms)
val_data = DataSet(val_df,'../data/goodreads-best-books/images/images/', transform = test_transforms)
test_data = DataSet(test_df,'../data/goodreads-best-books/images/images/', transform = test_transforms)

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size,
                                                shuffle = True, num_workers=mp.cpu_count())
val_data_loader = torch.utils.data.DataLoader(val_data, batch_size = batch_size,
                                              shuffle = True, num_workers=mp.cpu_count())
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size,
                                              shuffle = True, num_workers=mp.cpu_count())

In [10]:
vgg = torchvision.models.vgg11(pretrained=True)
best_vgg = torchvision.models.vgg11(pretrained=False)

for param in vgg.features.parameters():
    param.requires_grad = False
    
vgg.classifier[6] = nn.Linear(4096, len(classes))
best_vgg.classifier[6] = nn.Linear(4096, len(classes))

    
if train_on_gpu:
    vgg = vgg.cuda()
    if multiple_gpus:
        vgg = nn.DataParallel(vgg)

In [13]:
def get_img(img_path):
    img = cv2.imread(img_path)
    img = test_transforms(img).float()
    img = img.unsqueeze_(0)
    if torch.cuda.is_available():
        img = img.cuda()
    return img

In [14]:
vgg(get_img('../data/goodreads-best-books/images/images/1.jpg'))

tensor([[ 0.0808,  1.0713, -0.2789, -0.4834,  0.2332,  0.3934, -0.0468,  0.8295,
         -1.0715, -0.3493,  0.6807,  0.4725,  0.5263,  0.0692,  1.0798,  0.1507,
          0.3750,  1.2967, -0.0882, -0.3331,  0.3074,  1.0853,  0.4363, -0.1746,
         -0.3810, -0.3962, -0.5177, -0.1824, -0.3178,  0.1395,  0.2341, -0.1760,
          0.2337, -0.3395, -0.1082,  0.2335,  0.9745,  0.3983,  0.6020,  0.2967,
          0.4925, -0.5742, -0.0815,  0.1247, -0.1142, -0.2864,  0.2189, -0.2765,
          0.0837,  0.5383, -0.9976,  0.1351, -0.1051, -0.4067, -0.2261, -0.1449,
         -0.5838,  0.0169,  0.4973, -0.1678,  0.1134,  0.1673,  0.7619, -0.3302,
         -0.0060,  0.8333,  0.3904, -0.8989,  0.5593,  0.7424, -0.1790,  0.2037,
         -0.3888, -0.2287, -0.6974, -0.6220, -0.2491]], device='cuda:0',
       grad_fn=<GatherBackward>)

In [8]:
def train_one_epoch(model, optimizer, criterion, train_data_loader, val_data_loader):
    train_loss = 0
    val_loss = 0

    model.train()
    for images, labels in train_data_loader:

        if train_on_gpu:
            images = images.cuda()
            labels = labels.cuda()

        optimizer.zero_grad()
        out = model(images)
        loss = criterion(out, labels)

        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    model.eval()
    for images, labels in val_data_loader:

        if train_on_gpu:
            images = images.cuda()
            labels = labels.cuda()

        out = model(images)
        loss = criterion(out, labels)

        val_loss += loss.item()
    train_loss = train_loss/len(train_data_loader.dataset)
    val_loss = val_loss/len(val_data_loader.dataset)  
    print('Training Loss: {:.6f} \tValidation Loss: {:.6f}'.format(train_loss, val_loss))
    
    return train_loss, val_loss

In [9]:
def train_tune_model(model, optimizer, criterion, train_data_loader, val_data_loader, epochs, model_name):
    no_improvement=0
    best_loss = np.inf
    train_losses = []
    val_losses = []
    
    for i, epoch in enumerate(range(1, epochs+1)):
        train_loss, val_loss = train_one_epoch(model, optimizer, criterion, train_data_loader, val_data_loader)
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        p_count = 0
        if multiple_gpus:
            for p in model.module.features.named_parameters():
                if p_count >= 16-i*2:
                    p[1].requires_grad = True
                    print('Tracking Gradient For:', p[0])
                p_count += 1
                
        else:
            for p in model.features.named_parameters():
                if p_count >= 16-i*2:
                    p[1].requires_grad = True
                    print('Tracking Gradient For:', p[0])
                p_count += 1

        #Saving the weights of the best model according to validation score
        if val_loss < best_loss:
            no_improvement = 0
            best_loss = val_loss
            print('Improved Model Score - Updating Best Model Parameters...')
            if multiple_gpus:
                torch.save(model.module.state_dict(), f'../models/{model_name}.pt')
            else:
                torch.save(model.state_dict(), f'../models/{model_name}.pt')
        else:
            no_improvement +=1
            if no_improvement==5:
                print('No Improvement for 5 epochs, Early Stopping')
                break
    joblib.dump(train_losses, '../data/tuned_train_losses.pkl')
    joblib.dump(val_losses, '../data/tuned_val_losses.pkl')

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adamax(vgg.parameters(), lr=.0001)

train_tune_model(vgg, optimizer, criterion, train_data_loader, val_data_loader, 200, 'vgg_tuned')

Training Loss: 0.015777 	Validation Loss: 0.014643
Improved Model Score - Updating Best Model Parameters...
Training Loss: 0.013761 	Validation Loss: 0.014115
Tracking Gradient For: 18.weight
Tracking Gradient For: 18.bias
Improved Model Score - Updating Best Model Parameters...
Training Loss: 0.012932 	Validation Loss: 0.013661
Tracking Gradient For: 16.weight
Tracking Gradient For: 16.bias
Tracking Gradient For: 18.weight
Tracking Gradient For: 18.bias
Improved Model Score - Updating Best Model Parameters...
Training Loss: 0.012117 	Validation Loss: 0.013450
Tracking Gradient For: 13.weight
Tracking Gradient For: 13.bias
Tracking Gradient For: 16.weight
Tracking Gradient For: 16.bias
Tracking Gradient For: 18.weight
Tracking Gradient For: 18.bias
Improved Model Score - Updating Best Model Parameters...
Training Loss: 0.011258 	Validation Loss: 0.013221
Tracking Gradient For: 11.weight
Tracking Gradient For: 11.bias
Tracking Gradient For: 13.weight
Tracking Gradient For: 13.bias
Track

In [11]:
best_vgg.load_state_dict(torch.load('../models/vgg_tuned.pt'))

best_vgg = best_vgg.cuda()
best_vgg.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_data_loader:
        if train_on_gpu:
            images = images.cuda()
            labels = labels.cuda()
        outputs = best_vgg(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
          
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 34.5276008492569 %
